# DOT-MATRIX PLOT
- For exploratory sequence matching
- Used for qualitative sequence alignment

In [0]:
def delta(x,y):
    return 0 if x == y else 1

def M(seq1,seq2,i,j,k):
    return sum(delta(x,y) for x,y in zip(seq1[i:i+k],seq2[j:j+k]))

def makeMatrix(seq1,seq2,k):
    n = len(seq1)
    m = len(seq2)
    return [[M(seq1,seq2,i,j,k) for j in range(m-k+1)] for i in range(n-k+1)]

def plotMatrix(M,t, seq1, seq2, nonblank = chr(0x25A0), blank = ' '):
    print(' |' + seq2)
    print('-'*(2 + len(seq2)))
    for label,row in zip(seq1,M):
        line = ''.join(nonblank if s < t else blank for s in row)
        print(label + '|' + line)
        
def dotplot(seq1,seq2,k = 1,t = 1):
    M = makeMatrix(seq1,seq2,k)
    plotMatrix(M, t, seq1,seq2)

In [0]:
seq1 = "AAAATATATATAGAGAGA"
seq2 = "GCGCATATACATGATTCA"
dotplot(seq1,seq2)

 |GCGCATATACATGATTCA
--------------------
A|    ■ ■ ■ ■  ■   ■
A|    ■ ■ ■ ■  ■   ■
A|    ■ ■ ■ ■  ■   ■
A|    ■ ■ ■ ■  ■   ■
T|     ■ ■   ■  ■■  
A|    ■ ■ ■ ■  ■   ■
T|     ■ ■   ■  ■■  
A|    ■ ■ ■ ■  ■   ■
T|     ■ ■   ■  ■■  
A|    ■ ■ ■ ■  ■   ■
T|     ■ ■   ■  ■■  
A|    ■ ■ ■ ■  ■   ■
G|■ ■         ■     
A|    ■ ■ ■ ■  ■   ■
G|■ ■         ■     
A|    ■ ■ ■ ■  ■   ■
G|■ ■         ■     
A|    ■ ■ ■ ■  ■   ■


# Needleman-Wunsch Algorithm

In [0]:
def zeros(rows, cols):
    zeromat = []
    for x in range(rows):
        zeromat.append([])
        for y in range(cols):
            zeromat[-1].append(0)
    return zeromat

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

def needleman_wunsch_Matrix(seq1, seq2):
    
    # length of two sequences
    n,m= len(seq2),len(seq1) 
    
    # Generate matrix of zeros to store scores
    score = zeros(m+1, n+1)
    
    # Fill out first column
    for i in range(0, m + 1):
        score[i][0] = gap_penalty * i
    # Fill out first row
    for j in range(0, n + 1):
        score[0][j] = gap_penalty * j
    
    # Fill out all other values in the score matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Calculate the score by checking the top, left, and diagonal cells
            match = score[i - 1][j - 1] + match_score(seq1[j-1], seq2[i-1])
            delete = score[i - 1][j] + gap_penalty
            insert = score[i][j - 1] + gap_penalty
            # Record the maximum score from the three possible scores calculated above
            score[i][j] = max(match, delete, insert)

    return score


def needleman_wunsch_alignment(score): 
    # Create variables to store alignment
    align1 = ""
    align2 = ""
    # Start from the bottom right cell in matrix
    i = m
    j = n
    while i > 0 and j > 0:
        score_current = score[i][j]
        score_diagonal = score[i-1][j-1]
        score_up = score[i][j-1]
        score_left = score[i-1][j]
        
        # Check to figure out which cell the current score was calculated from,
        # then update i and j to correspond to that cell.
        if score_current == score_diagonal + match_score(seq1[j-1], seq2[i-1]):
            align1 += seq1[j-1]
            align2 += seq2[i-1]
            i -= 1
            j -= 1
        elif score_current == score_up + gap_penalty:
            align1 += seq1[j-1]
            align2 += '-'
            j -= 1
        elif score_current == score_left + gap_penalty:
            align1 += '-'
            align2 += seq2[i-1]
            i -= 1
    # Finish tracing up to the top left cell
    while j > 0:
        align1 += seq1[j-1]
        align2 += '-'
        j -= 1
    while i > 0:
        align1 += '-'
        align2 += seq2[i-1]
        i -= 1
    
    # Since we traversed the score matrix from the bottom right, our two sequences will be reversed.
    # These two lines reverse the order of the characters in each sequence.
    align1 = align1[::-1]
    align2 = align2[::-1]
    
    return(align1, align2)


In [13]:
import numpy as np
gap_penalty = -1
mismatch_penalty = -1
match_award = 2

print(np.matrix(needleman_wunsch_Matrix(seq1, seq2)))
output1, output2 = needleman_wunsch(seq1, seq2)
print(output1 + "\n" + output2)

[[  0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17
  -18]
 [ -1  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -10 -11 -12 -13 -14
  -15]
 [ -2  -2  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -11 -11 -12 -13 -14
  -15]
 [ -3  -3  -3  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -10 -11  -9 -10 -11
  -12]
 [ -4  -4  -4  -4  -4  -5  -6  -7  -8  -9 -10 -11 -12 -11 -11 -10 -10 -11
  -12]
 [ -5  -2  -2  -2  -2  -3  -3  -4  -5  -6  -7  -8  -9 -10  -9 -10  -8  -9
   -9]
 [ -6  -3  -3  -3  -3   0  -1  -1  -2  -3  -4  -5  -6  -7  -8  -9  -9  -9
  -10]
 [ -7  -4  -1  -1  -1  -1   2   1   1   0  -1  -2  -3  -4  -5  -6  -7  -8
   -7]
 [ -8  -5  -2  -2  -2   1   1   4   3   3   2   1   0  -1  -2  -3  -4  -5
   -6]
 [ -9  -6  -3   0   0   0   3   3   6   5   5   4   3   2   1   0  -1  -2
   -3]
 [-10  -7  -4  -1  -1  -1   2   2   5   5   4   4   3   2   1   0  -1  -2
   -3]
 [-11  -8  -5  -2   1   0   1   1   4   4   7   6   6   5   4   3   2   1
    0]
 [-12  -9  -6  -3   0   3   

# Smith-Waterman Algorithm

In [0]:
def SW_ALGO(s1, s2):
    m, n = len(s1), len(s2)
    V = np.zeros((m+1, n+1))    
    T = np.zeros((m+1, n+1))
    max_score = 0
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            sc_diag = V[i-1][j-1] + match_score(s1[i-1], s2[j-1])
            sc_up = V[i][j-1] +  gap_penalty
            sc_left = V[i-1][j] +  gap_penalty
            V[i][j] = max(0,sc_left, sc_up, sc_diag)
            if V[i][j] == 0: T[i][j] = 0
            if V[i][j] == sc_left: T[i][j] = 1
            if V[i][j] == sc_up: T[i][j] = 2
            if V[i][j] == sc_diag: T[i][j] = 3
            if V[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = V[i][j];
    
    
    print('V=\n',V,'\n')
    align1, align2 = '', ''
    i,j = max_i,max_j
    
    
    while T[i][j] != 0:
        if T[i][j] == 3:
            a1 = s1[i-1]
            a2 = s2[j-1]
            i -= 1
            j -= 1
        elif T[i][j] == 2:
            a1 = '-'
            a2 = s2[j-1]
            j -= 1
        elif T[i][j] == 1:
            a1 = s1[i-1]
            a2 = '-'
            i -= 1
        print('a1 = %s\t a2 = %s\n' % (a1,a2))
        align1 += a1
        align2 += a2
        
        

    align1 = align1[::-1]
    align2 = align2[::-1]
    sym = ''
    iden = 0
    
    
    for i in range(len(align1)):
        a1 = align1[i]
        a2 = align2[i]
        if a1 == a2:                
            sym += a1
            iden += 1
        elif a1 != a2 and a1 != '-' and a2 != '-': 
            sym += ' '
        elif a1 == '-' or a2 == '-':          
            sym += ' '
            
            
    
    identity = iden / len(align1) * 100
    print('Identity = %f percent' % identity)
    print('Score =', max_score)
    print(align1)
    print(sym)
    print(align2)

In [32]:
SW_ALGO(seq1,seq2)

V=
 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  0.  2.  1.  2.  1.  2.  1.  2.  1.  0.  2.  1.  0.  0.
   2.]
 [ 0.  0.  0.  0.  0.  2.  1.  3.  2.  3.  2.  3.  2.  1.  2.  1.  0.  0.
   2.]
 [ 0.  0.  0.  0.  0.  2.  1.  3.  2.  4.  3.  4.  3.  2.  3.  2.  1.  0.
   2.]
 [ 0.  0.  0.  0.  0.  2.  1.  3.  2.  4.  3.  5.  4.  3.  4.  3.  2.  1.
   2.]
 [ 0.  0.  0.  0.  0.  1.  4.  3.  5.  4.  3.  4.  7.  6.  5.  6.  5.  4.
   3.]
 [ 0.  0.  0.  0.  0.  2.  3.  6.  5.  7.  6.  5.  6.  6.  8.  7.  6.  5.
   6.]
 [ 0.  0.  0.  0.  0.  1.  4.  5.  8.  7.  6.  5.  7.  6.  7. 10.  9.  8.
   7.]
 [ 0.  0.  0.  0.  0.  2.  3.  6.  7. 10.  9.  8.  7.  6.  8.  9.  9.  8.
  10.]
 [ 0.  0.  0.  0.  0.  1.  4.  5.  8.  9.  9.  8. 10.  9.  8. 10. 11. 10.
   9.]
 [ 0.  0.  0.  0.  0.  2.  3.  6.  7. 10.  9. 11. 10.  9. 11. 10. 10. 10.
  12.]
 [ 0.  0.  0.  0.  0.  1.  4.  5.  8.  9.  9. 10. 13. 12. 11. 13. 12. 11.
  11.]
 [ 0.  0.  0.  0.  0.  2